In [9]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

df_train = pd.read_csv('train_tweets.csv')
print(df_train.head(10))

print("\n\n\n")

df_test = pd.read_csv("test_tweets.csv")
print(df_test.head(10))


df_mixed_train = pd.read_csv("train_tweets_mixed.csv")
print(df_test.head(10))

                                               tweet sentiment
0  trt1 e bak beni bloklamış benim vergimle beni ...   Negatif
1                      sabah sabah siktirin gidin ya   Negatif
2                        Bu adam senimi sikdi bummm    Negatif
3   Katliamdan kaçan Suriye liler gitsin Suriye d...   Negatif
4                    gercekten pisliginde bogul amk    Negatif
5                        Şık bir hareket olur Brate    Pozitif
6  Bilginliği ile hava atmaya çalışan gerzek bir ...   Negatif
7  Yine kazık gibi oldu var mı kucağa gelmek iste...   Negatif
8  nerde salak nerde gerizeka varsa elimle koymuş...   Negatif
9                                nerdesin len tirrek   Negatif




                                               tweet sentiment
0  Anne bir sanatçıdır, en güzel eseri de yavrusu...   Pozitif
1                Ben söğüşledim, birazda sen söğüşle   Negatif
2  Şerefsizlik, sözde sanatçıların vazgeçemediği ...   Negatif
3              Kendisi de bilmiyordur çünkü beyinsi

In [10]:
x_train = df_train.tweet
y_train = df_train.sentiment

x_test= df_test.tweet
y_test = df_test.sentiment

x_mixed_train = df_mixed_train.tweet
y_mixed_train = df_mixed_train.sentiment

y_test = df_test.sentiment



In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, base ------")
my_labels = ["Pozitif", "Negatif"]
print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_base_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_base_report)

------Naive Bayes, base ------
accuracy 0.8472824274758253
              precision    recall  f1-score   support

     Pozitif       0.86      0.78      0.82      1309
     Negatif       0.84      0.90      0.87      1690

    accuracy                           0.85      2999
   macro avg       0.85      0.84      0.84      2999
weighted avg       0.85      0.85      0.85      2999



In [12]:
df_mixed_default_train = pd.read_csv("train_default_tweets_mixed.csv")


x_mixed_default_train = df_mixed_default_train.tweet
y_mixed_default_train = df_mixed_default_train.sentiment
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_default_train, y_mixed_default_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, data augmented with default gpt2------")

print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_default_augmented_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_default_augmented_report)


------Naive Bayes, data augmented with default gpt2------
accuracy 0.7899299766588863
              precision    recall  f1-score   support

     Pozitif       0.97      0.53      0.69      1309
     Negatif       0.73      0.99      0.84      1690

    accuracy                           0.79      2999
   macro avg       0.85      0.76      0.77      2999
weighted avg       0.84      0.79      0.77      2999



In [13]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_train, y_mixed_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, data augmented with our model------")

print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_augmented_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_augmented_report)


------Naive Bayes, data augmented with our model------
accuracy 0.7985995331777259
              precision    recall  f1-score   support

     Pozitif       0.98      0.55      0.70      1309
     Negatif       0.74      0.99      0.85      1690

    accuracy                           0.80      2999
   macro avg       0.86      0.77      0.78      2999
weighted avg       0.84      0.80      0.79      2999



In [14]:
df_mixed_rlhf_train = pd.read_csv("train_tweets_mixed_rlhf.csv")
x_mixed_rlhf_train = df_mixed_rlhf_train.tweet
y_mixed_rlhf_train = df_mixed_rlhf_train.sentiment

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_rlhf_train, y_mixed_rlhf_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, data augmented with our model built with RLHF------")

print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_rlhf_augmented_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_augmented_report)


------Naive Bayes, data augmented with our model built with RLHF------
accuracy 0.8109369789929977
              precision    recall  f1-score   support

     Pozitif       0.98      0.55      0.70      1309
     Negatif       0.74      0.99      0.85      1690

    accuracy                           0.80      2999
   macro avg       0.86      0.77      0.78      2999
weighted avg       0.84      0.80      0.79      2999



In [19]:
print("base\n")
print(NB_base_report)
print("\n------------------------------------------\n")
print("data augmented with baseline gpt2")
print(NB_default_augmented_report)
print("\n------------------------------------------\n")

print("data augmented with pre-trained sentiment analysis model")
print(NB_augmented_report)
print("\n------------------------------------------\n")

print("data augmented with our human-feedback trained model")
print(NB_rlhf_augmented_report)


base

              precision    recall  f1-score   support

     Pozitif       0.86      0.78      0.82      1309
     Negatif       0.84      0.90      0.87      1690

    accuracy                           0.85      2999
   macro avg       0.85      0.84      0.84      2999
weighted avg       0.85      0.85      0.85      2999


------------------------------------------

data augmented with baseline gpt2
              precision    recall  f1-score   support

     Pozitif       0.97      0.53      0.69      1309
     Negatif       0.73      0.99      0.84      1690

    accuracy                           0.79      2999
   macro avg       0.85      0.76      0.77      2999
weighted avg       0.84      0.79      0.77      2999


------------------------------------------

data augmented with pre-trained sentiment analysis model
              precision    recall  f1-score   support

     Pozitif       0.98      0.55      0.70      1309
     Negatif       0.74      0.99      0.85      1